In [6]:
import sys
sys.path.insert(0, "/home/uih/JYL/Programs/YOLO/")
import torch
from models import YoloXSmall, Yolov5Small, Yolov5SmallWithPlainBscp, Yolov5Middle

In [7]:
class_num = 1

In [8]:
# yolo = Yolov5Small(3, class_num)
yolo = YoloXSmall(num_anchors=1, in_channel=3, num_classes=class_num)
# yolo = Yolov5SmallWithPlainBscp(3, class_num)
# yolo = Yolov5Middle(3, class_num)
my_state_dict = yolo.state_dict()

In [9]:
my_total_k = []
my_total_v = []
my_focus_layer = []
my_stage_1_conv = []
for i, (k, v) in enumerate(my_state_dict.items()):
    my_total_k.append(k)
    my_total_v.append(v)
    print(f'{i}: {k}\t\t{v.shape}')
    if "neck" in k and "focus" in k:
        my_focus_layer.append(v)
        continue
    if "neck" in k and "stage1_conv" in k:
        my_stage_1_conv.append(v)

0: neck.focus.conv.weight		torch.Size([32, 3, 6, 6])
1: neck.focus.bn.weight		torch.Size([32])
2: neck.focus.bn.bias		torch.Size([32])
3: neck.focus.bn.running_mean		torch.Size([32])
4: neck.focus.bn.running_var		torch.Size([32])
5: neck.focus.bn.num_batches_tracked		torch.Size([])
6: neck.backbone_stage1_conv.conv.weight		torch.Size([64, 32, 3, 3])
7: neck.backbone_stage1_conv.bn.weight		torch.Size([64])
8: neck.backbone_stage1_conv.bn.bias		torch.Size([64])
9: neck.backbone_stage1_conv.bn.running_mean		torch.Size([64])
10: neck.backbone_stage1_conv.bn.running_var		torch.Size([64])
11: neck.backbone_stage1_conv.bn.num_batches_tracked		torch.Size([])
12: neck.backbone_stage1_bscp.cba1.conv.weight		torch.Size([32, 64, 1, 1])
13: neck.backbone_stage1_bscp.cba1.bn.weight		torch.Size([32])
14: neck.backbone_stage1_bscp.cba1.bn.bias		torch.Size([32])
15: neck.backbone_stage1_bscp.cba1.bn.running_mean		torch.Size([32])
16: neck.backbone_stage1_bscp.cba1.bn.running_var		torch.Size([32])
17: n

In [10]:
v5_state_dict = torch.load('/home/uih/JYL/Programs/YOLO_ckpts/yolov5_small_for_voc.pth', map_location='cpu')

In [11]:
v5_state_dict = v5_state_dict['model_state_dict']

In [12]:
for i, (k, v) in enumerate(v5_state_dict.items()):
    print(k, v.shape)

focus.conv.weight torch.Size([32, 3, 6, 6])
focus.bn.weight torch.Size([32])
focus.bn.bias torch.Size([32])
focus.bn.running_mean torch.Size([32])
focus.bn.running_var torch.Size([32])
focus.bn.num_batches_tracked torch.Size([])
backbone_stage1_conv.conv.weight torch.Size([64, 32, 3, 3])
backbone_stage1_conv.bn.weight torch.Size([64])
backbone_stage1_conv.bn.bias torch.Size([64])
backbone_stage1_conv.bn.running_mean torch.Size([64])
backbone_stage1_conv.bn.running_var torch.Size([64])
backbone_stage1_conv.bn.num_batches_tracked torch.Size([])
backbone_stage1_bscp.cba1.conv.weight torch.Size([32, 64, 1, 1])
backbone_stage1_bscp.cba1.bn.weight torch.Size([32])
backbone_stage1_bscp.cba1.bn.bias torch.Size([32])
backbone_stage1_bscp.cba1.bn.running_mean torch.Size([32])
backbone_stage1_bscp.cba1.bn.running_var torch.Size([32])
backbone_stage1_bscp.cba1.bn.num_batches_tracked torch.Size([])
backbone_stage1_bscp.cba2.conv.weight torch.Size([32, 64, 1, 1])
backbone_stage1_bscp.cba2.bn.weight 

In [13]:
v5_total_k, v5_total_v = [], []
v5_focus_layer = []
v5_stage_1_conv = []
for i, (k, v) in enumerate(v5_state_dict.items()):
    if 'anchor' not in k:
        v5_total_k.append(k)
        v5_total_v.append(v)
        print(f'{i}: {k}\t\t{v.shape}')
    if "model.0" in k:
        v5_focus_layer.append(v)
    if "model.1" in k:
        v5_stage_1_conv.append(v)

0: focus.conv.weight		torch.Size([32, 3, 6, 6])
1: focus.bn.weight		torch.Size([32])
2: focus.bn.bias		torch.Size([32])
3: focus.bn.running_mean		torch.Size([32])
4: focus.bn.running_var		torch.Size([32])
5: focus.bn.num_batches_tracked		torch.Size([])
6: backbone_stage1_conv.conv.weight		torch.Size([64, 32, 3, 3])
7: backbone_stage1_conv.bn.weight		torch.Size([64])
8: backbone_stage1_conv.bn.bias		torch.Size([64])
9: backbone_stage1_conv.bn.running_mean		torch.Size([64])
10: backbone_stage1_conv.bn.running_var		torch.Size([64])
11: backbone_stage1_conv.bn.num_batches_tracked		torch.Size([])
12: backbone_stage1_bscp.cba1.conv.weight		torch.Size([32, 64, 1, 1])
13: backbone_stage1_bscp.cba1.bn.weight		torch.Size([32])
14: backbone_stage1_bscp.cba1.bn.bias		torch.Size([32])
15: backbone_stage1_bscp.cba1.bn.running_mean		torch.Size([32])
16: backbone_stage1_bscp.cba1.bn.running_var		torch.Size([32])
17: backbone_stage1_bscp.cba1.bn.num_batches_tracked		torch.Size([])
18: backbone_stage1_b

In [14]:
len(my_total_k), len(v5_total_k)

(414, 348)

In [15]:
for i in range(min(len(my_total_k), len(v5_total_k))):
    if my_total_v[i].shape != v5_total_v[i].shape:
        print(i)

342
343
344
345
346
347


## load prarmeters

In [16]:
i = 0
total = v5_total_v[:]
for m in yolo.modules():
    if i >= 342:  # 只load两个网络间相同layer的参数，其他参数使用默认初始化
        continue
    if isinstance(m, torch.nn.Conv2d):
        m.weight.data = total[i]
        print(m.weight.data.shape, total[i].shape)
        i += 1
        if m.bias is not None:
            m.bias.data = total[i]
            i += 1
        
    if isinstance(m, torch.nn.BatchNorm2d):
        m.weight.data = total[i]
        i += 1
        m.bias.data = total[i]
        i += 1
        m.running_mean = total[i]
        i += 1
        m.running_var = total[i]
        i += 1
        m.num_batches_tracked = total[i]
        i += 1

torch.Size([32, 3, 6, 6]) torch.Size([32, 3, 6, 6])
torch.Size([64, 32, 3, 3]) torch.Size([64, 32, 3, 3])
torch.Size([32, 64, 1, 1]) torch.Size([32, 64, 1, 1])
torch.Size([32, 64, 1, 1]) torch.Size([32, 64, 1, 1])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([32, 32, 1, 1]) torch.Size([32, 32, 1, 1])
torch.Size([32, 32, 3, 3]) torch.Size([32, 32, 3, 3])
torch.Size([128, 64, 3, 3]) torch.Size([128, 64, 3, 3])
torch.Size([64, 128, 1, 1]) torch.Size([64, 128, 1, 1])
torch.Size([64, 128, 1, 1]) torch.Size([64, 128, 1, 1])
torch.Size([128, 128, 1, 1]) torch.Size([128, 128, 1, 1])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3]) torch.Size([64, 64, 3, 3])
torch.Size([64, 64, 1, 1]) torch.Size([64, 64, 1, 1])
torch.Size([64, 64, 3, 3]) torch.Size([64, 64, 3, 3])
torch.Size([256, 128, 3, 3]) torch.Size([256, 128, 3, 3])
torch.Size([128, 256, 1, 1]) torch.Size([128, 256, 1, 1])
torch.Size([128, 256, 1, 1]) torch.Size([128, 256, 1, 1])
torch.Si

In [17]:
i

342

## check

In [18]:
i = 0
total = v5_total_v[:]
for m in yolo.modules():
    if i >= 342:  # 只检查上一步load参数是否一致
        continue
    if isinstance(m, torch.nn.Conv2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        if m.bias is not None:
            assert (m.bias.data == total[i]).all()
            i += 1
    if isinstance(m, torch.nn.BatchNorm2d):
        assert (m.weight.data == total[i]).all()
        i += 1
        assert (m.bias.data == total[i]).all()
        i += 1
        assert (m.running_mean == total[i]).all()
        i += 1
        assert (m.running_var == total[i]).all()
        i += 1
        assert (m.num_batches_tracked == total[i]).all()
        i += 1

In [19]:
my_output_state = yolo.state_dict()
torch.save({'model_state_dict': my_output_state}, "/home/uih/JYL/Programs/YOLO_ckpts/yolox_small_for_wheat.pth")